# Example to invoke DLP to Analys a BQ Table

#### STEP 1 -  Install package



In [ ]:
!pip install google-cloud
!pip install google-api-python-client
!pip install oauth2client
!pip install google-cloud-bigquery
!pip install google-cloud-dlp

#### STEP 2 -  Set properties

In [ ]:
SERVICE_ACCOUNT='jupytersa'
JSON_FILE_NAME = 'sa_bq.json'
GCP_PROJECT_ID='your_project'
DATASET_ID = 'your_dataset'
TABLE_ID = 'your_table'

#### STEP 3 -  Load packages and Imports

In [ ]:
import logging
import json
import os
import pprint

from datetime import datetime
from googleapiclient import discovery
from google.cloud import bigquery
from google.cloud import dlp_v2
from oauth2client.service_account import ServiceAccountCredentials

#### STEP 4 -  Invoke DLP

In [ ]:
client_dlp = dlp_v2.DlpServiceClient.from_service_account_json(JSON_FILE_NAME)


inspect_job_data = {
    'storage_config': {
        'big_query_options': {
            'table_reference': {
                'project_id': GCP_PROJECT_ID,
                'dataset_id': DATASET_ID,
                'table_id': TABLE_ID
            },
            'rows_limit':10000,
            'sample_method':'RANDOM_START',
        },
    },
    'inspect_config': {
        'info_types': [
            {'name': 'ALL_BASIC'},
        ],
    },
    'actions': [
        {
            'save_findings': {
                'output_config':{
                    'table':{
                        'project_id': GCP_PROJECT_ID,
                        'dataset_id': DATASET_ID,
                        'table_id': '{}_DLP'.format(TABLE_ID)
                    }
                }
                
            },
        },
    ]
}

operation = client_dlp.create_dlp_job(parent=client_dlp.project_path(GCP_PROJECT_ID), inspect_job=inspect_job_data)

In [ ]:
xpto = client_dlp.get_dlp_job(operation.name)
print(xpto.state)

#### STEP 5 - Analysis Result in BQ

Wait some seconds to send the analys result to BQ


In [ ]:
client_bq = bigquery.Client.from_service_account_json(JSON_FILE_NAME)

# Perform a query.
QUERY = (
    'WITH result AS ('
    'SELECT'
    ' c1.info_type.name,'
    ' c1.likelihood,'
    ' content_locations.record_location.record_key.big_query_key.table_reference as bq,'
    ' content_locations.record_location.field_id as column '
    'FROM '
    ' `'+ GCP_PROJECT_ID +'.'+  DATASET_ID +'.'+  TABLE_ID  +'_DLP` as c1 '
    'CROSS JOIN UNNEST(c1.location.content_locations) AS content_locations '
    'WHERE c1.likelihood in (\'LIKELY\',\'VERY_LIKELY\'))'
    'SELECT r.name as info_type, r.likelihood, r.bq.project_id, r.bq.dataset_id,'
    ' r.bq.table_id, r.column.name, count(*) as count  FROM result r GROUP By 1,2,3,4,5,6 '
    'ORDER By COUNT DESC'
)
query_job = client_bq.query(QUERY)  # API request
rows = query_job.result() 
for row in rows:
    print('RULES: {} ({}) | COLUMN: {}.{}.{}:{} | count->{}'.format
          (row.info_type, row.likelihood, row.project_id,row.dataset_id,row.table_id,row.name, row.count))